<a href="https://colab.research.google.com/github/BasmaSayah/AraBERT-and-mBert-Insights-from-Psycholinguistic-Diagnostics/blob/main/code/aff_simp_mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install panda

  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=536e3d38d2ed5335a4fe056aec15c412f78531c5ceab683adb327b07cdb202c7
  Stored in directory: /root/.cache/pip/wheels/0e/8b/c3/ff9cbde1fffd8071cff8367a86f0350a1ce30a8d31b6a432e9
Successfully built panda


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00


In [ ]:
pip install torch

In [ ]:
pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=98d300f393b719f2786c0d4fe5516b3b720776f5e275a63be0695e5f2594c1c9
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


# **A function that predicts missing words in a text and returns the predictions and their corresponding probabilities in a list**

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from google.colab import drive
import os

# functions
def missingWordPredict(text):
  tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
  model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")
  # Tokenize text
  input_ids = tokenizer.encode(text, return_tensors='pt')
  # Find the masked token index
  mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
  # Generate predictions for masked token
  with torch.no_grad():
      outputs = model(input_ids)
      predictions = outputs.logits
  # Get the top 5 predictions and their scores
  top_predictions = torch.topk(predictions[0, mask_token_index], 5, dim=1).indices[0].tolist()
  predicted_token_indices = torch.topk(predictions[0, mask_token_index], 5).indices.tolist()
  predicted_probabilities = torch.nn.functional.softmax(predictions[0, mask_token_index[0]][predicted_token_indices].clone().detach(), dim=0).tolist()
  result=[]
  for i in range(5):
      predicted_token = tokenizer.decode([top_predictions[i]])
      result.append(predicted_token)
      result.append(predicted_probabilities[i])
  return result

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/phd/diagnosisFiles')
# loading the neg file into the dataframe df
df= pd.read_csv('neg', delimiter='\t')
df2= df.copy()
print(df2)

Mounted at /content/drive
                       السياق الخبري                          السياق المنفي  \
0   السلمون المرقط من مجموعة [MASK]   السلمون المرقط  ليس من مجموعة [MASK]    
1          السلمون من مجموعة [MASK]           السلمون ليس من مجموعة [MASK]    
2           النملة من مجموعة  [MASK]          النملة ليست من مجموعة [MASK]    
3           النحلة من مجموعة [MASK]            النحلة ليست من مجموعة [MASK]   
4     أبو الحنَّاء من مجموعة [MASK]      أبو الحنَّاء ليس من مجموعة [MASK]    
5           العصفور من مجموعة [MASK]           العصفور ليس من مجموعة [MASK]   
6           البلوط من مجموعة [MASK]            البلوط ليس من مجموعة [MASK]    
7          الصنوبر من مجموعة [MASK]            الصنوبر ليس من مجموعة [MASK]   
8           الوردة من مجموعة [MASK]            الوردة ليست من مجموعة [MASK]   
9          الأقحوان من مجموعة [MASK]          الأقحوان ليس من مجموعة [MASK]   
10          الجزرة  من مجموعة [MASK]           الجزرة ليست من مجموعة [MASK]   
11       البازلاء  من مجمو

# **Predicting missing words in affirmative sentences**

In [ ]:
result=[]
s= 'السلمون المرقط هو من [MASK]'
df2['p1']=None
df2['s1']=None
df2['p2']=None
df2['s2']=None
df2['p3']=None
df2['s3']=None
df2['p4']=None
df2['s4']=None
df2['p5']=None
df2['s5']=None
df2.columns


Index(['السياق الخبري', 'السياق المنفي', 'الهدف الخبري', 'الهدف المنفي', 'p1',
       's1', 'p2', 's2', 'p3', 's3', 'p4', 's4', 'p5', 's5'],
      dtype='object')

In [ ]:
for index, row in df2.iterrows():
  result=[]
  result= missingWordPredict(row["السياق الخبري"])
  row["p1"]=result[0]
  row["s1"]=result[1]
  row["p2"]=result[2]
  row["s2"]=result[3]
  row["p3"]=result[4]
  row["s3"]=result[5]
  row["p4"]=result[6]
  row["s4"]=result[7]
  row["p5"]=result[8]
  row["s5"]=result[9]
print(df2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the c

                       السياق الخبري                          السياق المنفي  \
0   السلمون المرقط من مجموعة [MASK]   السلمون المرقط  ليس من مجموعة [MASK]    
1          السلمون من مجموعة [MASK]           السلمون ليس من مجموعة [MASK]    
2           النملة من مجموعة  [MASK]          النملة ليست من مجموعة [MASK]    
3           النحلة من مجموعة [MASK]            النحلة ليست من مجموعة [MASK]   
4     أبو الحنَّاء من مجموعة [MASK]      أبو الحنَّاء ليس من مجموعة [MASK]    
5           العصفور من مجموعة [MASK]           العصفور ليس من مجموعة [MASK]   
6           البلوط من مجموعة [MASK]            البلوط ليس من مجموعة [MASK]    
7          الصنوبر من مجموعة [MASK]            الصنوبر ليس من مجموعة [MASK]   
8           الوردة من مجموعة [MASK]            الوردة ليست من مجموعة [MASK]   
9          الأقحوان من مجموعة [MASK]          الأقحوان ليس من مجموعة [MASK]   
10          الجزرة  من مجموعة [MASK]           الجزرة ليست من مجموعة [MASK]   
11       البازلاء  من مجموعة [MASK]         البازلاء

# **Calcuate accuracy when k=1 (when the number of predictions equals one)**




In [ ]:
#df2['in k1']=df2['الهدف الخبري'].str.contains('|'.join(df2['p1']), na=False)
df2['in k1'] = df2.apply(lambda x: x['p1'] in x['الهدف الخبري'], axis=1)

print(df2['الهدف الخبري'])
print(df2['p1'])
print(df2['in k1'])
num_answer_in_k1 = (df2['in k1'] == 1).sum()
average_column = df2['in k1'].mean()
print(average_column)

0                                         السمك|الأسماك
1                                         السمك|الأسماك
2                                          الحشرات|حشرة
3                                          الحشرات|حشرة
4                                           الطيور|طائر
5                                           الطيور|طائر
6                                          شجرة|الأشجار
7                                          شجرة|الأشجار
8                                           الزهور|زهرة
9                                           الزهور|زهرة
10                                          الخضار|خضار
11                                          الخضار|خضار
12                                         أدات|الأدوات
13                                         أدات|الأدوات
14                  |وسائل نقل|وسيلة نقل|المركبات|مركبة
15                   وسائل نقل|وسيلة نقل|المركبات|مركبة
16    الهياكل المعمارية|هيكل معماري|مبنى|مباني|منشآت...
17    الهياكل المعمارية|هيكل معماري|مبنى|مباني|م

# **Calcuate accuracy when k=5 (when the number of predictions equals five)**

In [ ]:
#df2['in k1']=df2['الهدف الخبري'].str.contains('|'.join(df2['p1']), na=False)
df2['in k2'] = df2.apply(lambda x: x['p2'] in x['الهدف الخبري'], axis=1)
df2['in k3'] = df2.apply(lambda x: x['p3'] in x['الهدف الخبري'], axis=1)
df2['in k4'] = df2.apply(lambda x: x['p4'] in x['الهدف الخبري'], axis=1)
df2['in k5'] = df2.apply(lambda x: x['p5'] in x['الهدف الخبري'], axis=1)
df2['in k1-5'] = df2[['in k1', 'in k2', 'in k3', 'in k4', 'in k5']].any(axis=1)
print(df2['الهدف الخبري'])
#print(df2['p1'])
print(df2['in k1-5'])
num_answer_in_k1 = (df2['in k1-5'] == 1).sum()
average_column = df2['in k1-5'].mean()
print(average_column)

0                                         السمك|الأسماك
1                                         السمك|الأسماك
2                                          الحشرات|حشرة
3                                          الحشرات|حشرة
4                                           الطيور|طائر
5                                           الطيور|طائر
6                                          شجرة|الأشجار
7                                          شجرة|الأشجار
8                                           الزهور|زهرة
9                                           الزهور|زهرة
10                                          الخضار|خضار
11                                          الخضار|خضار
12                                         أدات|الأدوات
13                                         أدات|الأدوات
14                  |وسائل نقل|وسيلة نقل|المركبات|مركبة
15                   وسائل نقل|وسيلة نقل|المركبات|مركبة
16    الهياكل المعمارية|هيكل معماري|مبنى|مباني|منشآت...
17    الهياكل المعمارية|هيكل معماري|مبنى|مباني|م

# **Negative sentences: predictions, probabilities, accuracy calculation**

In [ ]:
results=[]
df2['p1-n']=None
df2['s1-n']=None
df2['p2-n']=None
df2['s2-n']=None
df2['p3-n']=None
df2['s3-n']=None
df2['p4-n']=None
df2['s4-n']=None
df2['p5-n']=None
df2['s5-n']=None
df3= df2.copy()
df3.columns


Index(['السياق الخبري', 'السياق المنفي', 'الهدف الخبري', 'الهدف المنفي', 'p1',
       's1', 'p2', 's2', 'p3', 's3', 'p4', 's4', 'p5', 's5', 'in k1', 'in k2',
       'in k3', 'in k4', 'in k5', 'in k1-5', 'p1-n', 's1-n', 'p2-n', 's2-n',
       'p3-n', 's3-n', 'p4-n', 's4-n', 'p5-n', 's5-n'],
      dtype='object')

In [ ]:
for index, row in df2.iterrows():
    result = missingWordPredict(row["السياق المنفي"])

    if len(result) >= 10:  # Ensure the result list has enough elements
        df2.loc[index, 'p1-n'] = result[0]
        df2.loc[index, 's1-n'] = result[1]
        df2.loc[index, 'p2-n'] = result[2]
        df2.loc[index, 's2-n'] = result[3]
        df2.loc[index, 'p3-n'] = result[4]
        df2.loc[index, 's3-n'] = result[5]
        df2.loc[index, 'p4-n'] = result[6]
        df2.loc[index, 's4-n'] = result[7]
        df2.loc[index, 'p5-n'] = result[8]
        df2.loc[index, 's5-n'] = result[9]
    else:
        print("Insufficient elements in result list.")

print(df2['p1-n'])

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the c

0     .
1     .
2     .
3     .
4     .
5     .
6     .
7     .
8     .
9     .
10    .
11    .
12    .
13    .
14    .
15    .
16    .
17    .
Name: p1-n, dtype: object


In [ ]:
df2.columns

Index(['السياق الخبري', 'السياق المنفي', 'الهدف الخبري', 'الهدف المنفي', 'p1',
       's1', 'p2', 's2', 'p3', 's3', 'p4', 's4', 'p5', 's5', 'in k1', 'in k2',
       'in k3', 'in k4', 'in k5', 'in k1-5', 'p1-n', 's1-n', 'p2-n', 's2-n',
       'p3-n', 's3-n', 'p4-n', 's4-n', 'p5-n', 's5-n'],
      dtype='object')

In [ ]:
print(missingWordPredict(":السلمون المرقط من مجموعة [MASK]."))
print(missingWordPredict(":السلمون المرقط  ليس من مجموعة [MASK]."))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['C', 0.3143976628780365, 'X', 0.2192322313785553, ':', 0.18317295610904694, 'المجموعة', 0.15300352871418, 'أ', 0.13019365072250366]


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['واحدة', 0.31630629301071167, 'أخرى', 0.19604161381721497, 'X', 0.17325247824192047, 'المجموعة', 0.16692763566970825, 'C', 0.14747196435928345]


In [ ]:
#df2['in k1']=df2['الهدف الخبري'].str.contains('|'.join(df2['p1']), na=False)
df2['in k1-n'] = df2.apply(lambda x: x['p1-n'] in x['الهدف المنفي'], axis=1)

print(df2['الهدف المنفي'])
print(df2['p1-n'])
print(df2['in k1-n'])
num_answer_in_k1 = (df2['in k1-n'] == 1).sum()
average_column = df2['in k1-n'].mean()
print(average_column)

0       أداة
1       زهرة
2       خضار
3       مبنى
4       شجرة
5      سيارة
6      مركبة
7       أداة
8       حشرة
9       طائر
10      سمكة
11      بناء
12      حشرة
13    الخضار
14      شجرة
15      زهرة
16      سمكة
17      طائر
Name: الهدف المنفي, dtype: object
0     .
1     .
2     .
3     .
4     .
5     .
6     .
7     .
8     .
9     .
10    .
11    .
12    .
13    .
14    .
15    .
16    .
17    .
Name: p1-n, dtype: object
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
Name: in k1-n, dtype: bool
0.0


In [ ]:
df2['in k2-n'] = df2.apply(lambda x: x['p2-n'] in x['الهدف المنفي'], axis=1)
df2['in k3-n'] = df2.apply(lambda x: x['p3-n'] in x['الهدف المنفي'], axis=1)
df2['in k4-n'] = df2.apply(lambda x: x['p4-n'] in x['الهدف المنفي'], axis=1)
df2['in k5-n'] = df2.apply(lambda x: x['p5-n'] in x['الهدف المنفي'], axis=1)
df2['in k1-5-n'] = df2[['in k1-n', 'in k2-n', 'in k3-n', 'in k4-n', 'in k5-n']].any(axis=1)
print(df2['الهدف المنفي'])
#print(df2['p1'])
print(df2['in k1-5-n'])
num_answer_in_k1 = (df2['in k1-5-n'] == 1).sum()
average_column = df2['in k1-5-n'].mean()
print(average_column)

0       أداة
1       زهرة
2       خضار
3       مبنى
4       شجرة
5      سيارة
6      مركبة
7       أداة
8       حشرة
9       طائر
10      سمكة
11      بناء
12      حشرة
13    الخضار
14      شجرة
15      زهرة
16      سمكة
17      طائر
Name: الهدف المنفي, dtype: object
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
Name: in k1-5-n, dtype: bool
0.0


# **Download Results**

In [ ]:
from google.colab import files
#df2.to_csv('aff.tsv', index=False, sep='\t')
df2.to_csv('aff-neg-mbert.csv', index=False)
files.download('aff-neg-mbert.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>